## Installation

```sh
pip install coco-pano-ext-demo
```

In [1]:
from coco_pano_ext_demo import COCO, COCO_plot

## COCO Metrics in a bipartite graph framework

Even if the primary purpose of the COCO is to score segmentations, we use a graph framework and so we aim at scoring a bipartite graph P ↔ Q with degree 1 (1 node in P is paired with at most 1 node in Q).
The edge are weighted by shape matching metric the IoU. The figure below shows two segmentations and the corresponding bipartite graph to assess.

![](doc/matching.png)

![](doc/bipartite-5.png)